# KENV

<a href=mailto:fuodorov1998@gmail.com>V. Fedorov</a>, <a href=mailto:nikdanila@bk.ru>D. Nikiforov</a>, <a href=http://www.inp.nsk.su/~petrenko/>A. Petrenko</a>, (Novosibirsk, 2019)

## Manual

In [1]:
import kenv as kv

Сan call for help:

In [2]:
#help(kv)

## Simulation

To obtain an envelope, you need to create a beam and an accelerator. Then load the simulation of this beam in this accelerator and apply tracking.

In [3]:
#help(kv.Beam)

In [4]:
beam = kv.Beam(energy=2,
               current=2e3,
               radius=50e-3,
               angular=55.0e-3,
               normalized_emittans=1000e-6)

In [5]:
beam.add_particle('Center',30e-3,0, 0,0)
#kv.Beam.add_particle?

Can print:

In [6]:
print(beam)

Beam parameters:
	Current	2000.0 A
	Energy	2.0 MeV
	Radius x	50.0 mm
	Radius y	50.0 mm
	Angular x	55.0 mrad
	Angular y	55.0 mrad
	Normalized emittans x	1000.0 mm*mrad
	Normalized emittans y	1000.0 mm*mrad
	Particles:
	[ 30.0 mm, 0.0 mm, 0.0 mrad, 0.0 mrad, 0.0 rad, 'Center'],



In [7]:
#help(kv.Accelerator)

In [8]:
accelerator = kv.Accelerator(0.7, 15, 0.001)

In [9]:
Solenoids = [ 
    [ 0.95,       0.001,    'B_z(z).dat', 'Sol. 1' ],
    [ 2.1,        0.03,     'B_z(z).dat', 'Sol. 2' ],
    [ 2.9077,     0.001,    'B_z(z).dat', 'Sol. 3' ],
    [ 4.0024,     0.03,     'B_z(z).dat', 'Sol. 4' ],
    [ 5.642,      0.0254,   'B_z(z).dat', 'Sol. 5' ],
    [ 6.760,      0.001,    'B_z(z).dat', 'Sol. 6' ],
]

In [10]:
Accels = [
    [ 7.456,       -0.9,     'E_z(z).dat',  'Cavity 3'],
    [ 8.838,       -0.9,     'E_z(z).dat',  'Cavity 4'],
    [ 10.220,      -0.9,     'E_z(z).dat',  'Cavity 5'],
    [ 11.602,      -0.9,     'E_z(z).dat',  'Cavity 6'],
    [ 12.984,      -0.9,     'E_z(z).dat',  'Cavity 7'],
    [ 14.366,      -0.9,     'E_z(z).dat',  'Cavity 8'],    
]

In [11]:
for   z0, B0, filename, name in Solenoids:
    accelerator.Bz_beamline[name] = kv.Element(z0, B0, filename, name)
for   z0, E0, filename, name in Accels:
    accelerator.Ez_beamline[name] = kv.Element(z0, E0, filename, name)

In [12]:
print(accelerator)

Accelerator structure.
	Solenoids:
	[ 0.95000 m, 0.00100 T, 'B_z(z).dat', 'Sol. 1'],
	[ 2.10000 m, 0.03000 T, 'B_z(z).dat', 'Sol. 2'],
	[ 2.90770 m, 0.00100 T, 'B_z(z).dat', 'Sol. 3'],
	[ 4.00240 m, 0.03000 T, 'B_z(z).dat', 'Sol. 4'],
	[ 5.64200 m, 0.02540 T, 'B_z(z).dat', 'Sol. 5'],
	[ 6.76000 m, 0.00100 T, 'B_z(z).dat', 'Sol. 6'],
	Accelerating modules:
	[ 7.45600 m, -0.90000 Mv/m, 'E_z(z).dat', 'Cavity 3'],
	[ 8.83800 m, -0.90000 Mv/m, 'E_z(z).dat', 'Cavity 4'],
	[ 10.22000 m, -0.90000 Mv/m, 'E_z(z).dat', 'Cavity 5'],
	[ 11.60200 m, -0.90000 Mv/m, 'E_z(z).dat', 'Cavity 6'],
	[ 12.98400 m, -0.90000 Mv/m, 'E_z(z).dat', 'Cavity 7'],
	[ 14.36600 m, -0.90000 Mv/m, 'E_z(z).dat', 'Cavity 8'],
	Quadrupoles:



In [13]:
accelerator.compile()

In [14]:
#help(kv.Simulation)

In [15]:
simulation = kv.Simulation(beam, accelerator)

In [ ]:
#simulation.track(solver='solve_ivp', rtol=1e-8)

simulation.track(solver='odeint', rtol=1e-8)

## Graphic with holoviews

### matplotlib

In [ ]:
import holoviews as hv
hv.extension('matplotlib')

%opts Layout [tight=True]
%output size=250 backend='matplotlib' fig='png'

%opts Area Curve [aspect=3 show_grid=True]
%opts Area  (alpha=0.25)
%opts Curve (alpha=0.5)
%opts Area.Beam [aspect=3 show_grid=True] (color='red' alpha=0.3)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
dim_z  = hv.Dimension('z',  unit='m', range=(accelerator.start, accelerator.stop))
dim_z_zoom = hv.Dimension('z_zoom',  unit='m', range=(7, 8.0))
dim_Bz = hv.Dimension('Bz', unit='T', label='Bz')
dim_Ez = hv.Dimension('Ez', unit='MV/m', label='Ez')
dim_dEzdz = hv.Dimension('dEzdz', unit='$\mathrm{MV/m^2}$', label='$dE_z/dz$', range=(-10,+10))
dim_dEzdz_zoom = hv.Dimension('dEzdz_zoom', unit='$\mathrm{MV/m^2}$', label='$dE_z/dz$_zoom', range=(-0.3,+0.3))
dim_dBzdz = hv.Dimension('dBzdz', unit='T/m', label='$dB_z/dz$', range=(-0.22,+0.22))

dim_r = hv.Dimension('x, y', label="Lab x,y", unit='mm', range=(-50,50))
dim_angular = hv.Dimension('x, y', label="Lab x',y'", unit='mrad', range=(-50,50))
dim_phase = hv.Dimension('$\phi$', label="Larmor phase", unit='rad')
dim_r_larmor = hv.Dimension('x, y', label="Larmor x,y", unit='mm', range=(-50,50))
dim_angular_larmor = hv.Dimension('x, y', label="Larmor x',y'", unit='mrad', range=(-50,50))

In [ ]:
z_Ez = hv.Area((accelerator.parameter,accelerator.Ez(accelerator.parameter)), kdims=dim_z, vdims=dim_Ez)
z_dEzdz = hv.Area((accelerator.parameter,accelerator.dEzdz(accelerator.parameter)), kdims=dim_z, vdims=dim_dEzdz)
z_dEzdz_zoom = hv.Area((accelerator.parameter,accelerator.dEzdz(accelerator.parameter)), kdims=dim_z_zoom, vdims=dim_dEzdz_zoom)
z_Bz = hv.Area((accelerator.parameter,accelerator.Bz(accelerator.parameter)), kdims=dim_z, vdims=dim_Bz)
z_dBzdz = hv.Area((accelerator.parameter,accelerator.dBzdz(accelerator.parameter)), kdims=dim_z, vdims=dim_dBzdz)

z_r = hv.Area(((accelerator.parameter,simulation.envelope_x*1e3)), kdims=[dim_z], vdims=[dim_r], group='Beam')
z_x_particle = hv.Curve(((accelerator.parameter,simulation.particle_x['Center']*1e3)), kdims=[dim_z], vdims=[dim_r], group='Beam', label='x')
z_y_particle = hv.Curve(((accelerator.parameter,simulation.particle_y['Center']*1e3)), kdims=[dim_z], vdims=[dim_r], group='Beam', label='y')
z_angular_x_particle = hv.Curve(((accelerator.parameter,simulation.particle_angular_x['Center']*1e3)), kdims=[dim_z], vdims=[dim_angular], label="x'")
z_angular_y_particle = hv.Curve(((accelerator.parameter,simulation.particle_angular_y['Center']*1e3)), kdims=[dim_z], vdims=[dim_angular], label="y'")

z_phase = hv.Curve(((accelerator.parameter,simulation.particle_larmor_phase['Center'])), kdims=[dim_z], vdims=[dim_phase])

z_x_larmor_particle = hv.Curve(((accelerator.parameter,simulation.particle_larmor_x['Center']*1e3)), kdims=[dim_z], vdims=[dim_r_larmor], label='x')
z_y_larmor_particle = hv.Curve(((accelerator.parameter,simulation.particle_larmor_y['Center']*1e3)), kdims=[dim_z], vdims=[dim_r_larmor], label='y')
z_angular_x_larmor_particle = hv.Curve(((accelerator.parameter,simulation.particle_larmor_angular_x['Center']*1e3)), kdims=[dim_z], vdims=[dim_angular_larmor], label="x'")
z_angular_y_larmor_particle = hv.Curve(((accelerator.parameter,simulation.particle_larmor_angular_y['Center']*1e3)), kdims=[dim_z], vdims=[dim_angular_larmor], label="y'")


In [ ]:
(z_x_particle*z_y_particle+z_x_larmor_particle*z_y_larmor_particle+z_phase).cols(1)


In [ ]:
(z_angular_x_particle*z_angular_y_particle+z_angular_x_larmor_particle*z_angular_y_larmor_particle+z_phase).cols(1)

In [ ]:
(z_Bz + z_dBzdz + z_Ez + z_dEzdz + z_dEzdz_zoom).cols(1)